In [1]:
%load_ext lab_black

In [2]:
%config Completer.use_jedi = False

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import time
from tqdm.notebook import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
path = "input/"

In [5]:
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

In [6]:
numerical_cols = [f"cont{i}" for i in range(1, 15)]
target_col = "target"

for c in numerical_cols:
    prep = StandardScaler()
    train[c] = prep.fit_transform(train[[c]])
    test[c] = prep.transform(test[[c]])

X_train = train.drop(["id", "target"], axis=1)
y_train = train["target"]
X_test = test.drop("id", axis=1)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=7)

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    model = keras.Sequential([
        layers.Dense(1024, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='linear'),
    ])

    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    early_stopping = keras.callbacks.EarlyStopping(
        patience=10,
        min_delta=0.05,
        restore_best_weights=True,
    )

    model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        batch_size=30000,
        epochs=1000,
        callbacks=[early_stopping],
    )

    oof_train[valid_index] = model.predict(X_val).reshape(1, -1)[0]
    y_pred = model.predict(X_test).reshape(1, -1)[0]

    y_preds.append(y_pred)
    models.append(model)

Epoch 1/1000
8/8 [==============================] - 3s 379ms/step - loss: 41.2493 - root_mean_squared_error: 6.3555 - val_loss: 9.1889 - val_root_mean_squared_error: 3.0313
Epoch 2/1000
8/8 [==============================] - 3s 381ms/step - loss: 8.9020 - root_mean_squared_error: 2.9737 - val_loss: 3.2354 - val_root_mean_squared_error: 1.7987
Epoch 3/1000
8/8 [==============================] - 3s 424ms/step - loss: 3.9020 - root_mean_squared_error: 1.9738 - val_loss: 2.4752 - val_root_mean_squared_error: 1.5733
Epoch 4/1000
8/8 [==============================] - 4s 450ms/step - loss: 2.3217 - root_mean_squared_error: 1.5234 - val_loss: 2.4315 - val_root_mean_squared_error: 1.5593
Epoch 5/1000
8/8 [==============================] - 3s 418ms/step - loss: 2.0986 - root_mean_squared_error: 1.4479 - val_loss: 2.0022 - val_root_mean_squared_error: 1.4150
Epoch 6/1000
8/8 [==============================] - 4s 493ms/step - loss: 1.8837 - root_mean_squared_error: 1.3722 - val_loss: 1.7029 - val

In [ ]:
models[0].summary()

In [ ]:
print(f"CV: {mean_squared_error(y_train, oof_train, squared=False)}")

In [ ]:
y_pred = sum(y_preds) / len(y_preds)
submission['target'] = y_pred
submission.to_csv('submission.csv', index=False)
submission.head()